## **Machine learning notebook for FDD Project @ Vasakronan summer internship 2022**

In [32]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
directory = '../meter_obs_and_irr'

## Create machine learning model for the generated energy/day

In [ ]:
def create_energy_day_models():
        k = 0
        files = Path(directory).glob('**/*')
        for filepath in files:
                print(filepath)
                data = pd.read_csv(filepath, skipinitialspace=True)
                meter_id = data['metry_meter_id'][0]
                print("doing id: " + meter_id)
                data = data.dropna(how = 'any')
                data['date_time'] = pd.to_datetime(data['date_time'])
                data = data.groupby(data.date_time.dt.date).agg({"irradiance": "sum", "generated_energy": "sum"})
                data['month'] = pd.to_datetime(
                data.round({'irradiance': 1, 'generated_energy': 1})

                x = data[["irradiance", "month"]]
                y = np.asarray(data[["generated_energy"]])
                if len(y) == 0:
                        continue
                
                x_train, x_test, y_train, y_test = train_test_split(
                        x, y, test_size=0.1, random_state=13)

                model = RandomForestRegressor(max_features="auto", bootstrap=True,
                max_depth= 80,
                min_samples_leaf= 5,
                min_samples_split= 12,
                n_estimators= 100)
                model.fit(x_train,y_train)


                print(model.score(x_test, y_test))

                preds = model.predict(x_test)
                print(len(preds))

                for i in range(len(preds)):
                        print('hej')
                        print(preds[i])
                        print(y_test[i])
                        print("Generated div pred", y_test[i]/preds[i])

                pickle.dump(model, open(f'../per_meter_models_energy/model_{meter_id}_energy_per_day.pkl', 'wb'))
                k += 1

create_energy_day_models()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)
plt.plot(model.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

### unused code

In [ ]:

def create_effect_hour_models():
    k = 0
    files = Path(directory).glob('**/*')
    done_ml = Path('./').glob('**/*.pkl')
    model_list = []
    for model in done_ml:
        model_list.append(str(model))
    for filepath in files:
        data = pd.read_csv(filepath)
        meter_id = data['metry_meter_id'][0]

        if meter_id in ''.join(model_list):
            continue
        
        print("doing id: " + meter_id)
        data = data.dropna(how = 'any')
        #try:
        data['date_time'] = pd.to_datetime(data['date_time'])
        data['month'] = data['date_time'].dt.month
        data.round({'irradiance': 1, 'generated_energy': 1})
        # Create a train and test set
        x = data[["irradiance", "month"]]
        y = np.asarray(data[["generated_energy"]])

        if len(y) == 0:
            continue

        x_train, x_test, y_train, y_test = train_test_split(
                x, y, test_size=0.10)
        #    filtered_data = data.loc[(data['date_time'] >= '2021-04-01T00:00:00Z')
        #                & (data['date_time'] < '2021-11-01T00:00:00Z')]
        #except:
        #     filtered_data = data

        model = MLPRegressor(activation = 'tanh', alpha = 0.05, early_stopping = True, hidden_layer_sizes = (50, 100, 50), learning_rate = 'constant', max_iter = 500, solver = 'adam')

        #param_grid = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
        #'activation': ['relu','tanh','logistic'],
        #'alpha': [0.0001, 0.05],
        #'early_stopping': [True, False],
        #'learning_rate': ['constant','adaptive'],
        #'max_iter': [500],
        #'solver': ['adam']
        #}

        #gsc = GridSearchCV(
        #    model,
        #    param_grid,
        #    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=-1)

        #grid_result = gsc.fit(x_train, y_train)

        #best_params = grid_result.best_params_
        #print(best_params)
    
        # model = RidgeCV()

        # print(model.score(x_test_scaled, y_test_scaled))
        model.fit(x_train, y_train)
        print(model.score(x_test, y_test))
        k = k + 1
        pickle.dump(model, open(f'model_{meter_id}.pkl', 'wb'))
        #pickle.dump(scalerx, open(f'scalerx_{meter_id}.pkl', 'wb'))
        #pickle.dump(scalery, open(f'scalery_{meter_id}.pkl', 'wb'))
        

        # print(model.score(x_test, y_test))
        # pickle.dump(model, open(f'model_{meter_id}.pkl', 'wb'))

create_effect_hour_models()

In [ ]:
data = pd.read_csv('./sthlm_csv_test2.csv')
data = data.dropna(how = 'any')

def mlp_model(X, Y):

    estimator=MLPRegressor()


    param_grid = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
            'activation': ['relu','tanh','logistic'],
            'alpha': [0.0001, 0.05],
            'learning_rate': ['constant','adaptive'],
            'solver': ['adam']}

    gsc = GridSearchCV(
        estimator,
        param_grid,
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, Y)


    best_params = grid_result.best_params_
    print(best_params)

    best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], 
                            activation =best_params["activation"],
                            solver=best_params["solver"],
                            max_iter= 5000, n_iter_no_change = 200
                )

    scoring = {
            'abs_error': 'neg_mean_absolute_error',
            'squared_error': 'neg_mean_squared_error',
            'r2':'r2'}

    scores = cross_validate(best_mlp, X, Y, cv=10, scoring=scoring, return_train_score=True, return_estimator = True)
    return scores

mlp_model(data[["irradiance"]][:100],
np.asarray(data[["generated_energy"]][:100]))